### Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# Optimization attempt drop 'APPLICATION_TYPE'
# Optimization attempt drop 'APPLICATION_TYPE' & 'SPECIAL_CONSIDERATIONS'
application_df=application_df.drop(columns=["EIN","NAME","APPLICATION_TYPE","AFFILIATION","SPECIAL_CONSIDERATIONS"])

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
STATUS               2
INCOME_AMT           9
ASK_AMT           8747
IS_SUCCESSFUL        2
dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
class_count=application_df.CLASSIFICATION.value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2561        1
C2150        1
C1283        1
C4200        1
C2190        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ..?
replace_class=list(class_count[class_count<1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df=pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
application_df=application_df.merge(encode_df,left_index=True,right_index=True)
application_df=application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_CommunityServ,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y=application_df["IS_SUCCESSFUL"].values
X=application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense
       (units=hidden_nodes_layer1, input_dim=number_input_features, 
                          activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                2160      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 4,621
Trainable params: 4,621
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 70/804 [=>............................] - ETA: 1s - loss: 0.7031 - accuracy: 0.5271
Epoch 00001: saving model to checkpoints\weights.01.hdf5
176/804 [=====>........................] - ETA: 1s - loss: 0.6809 - accuracy: 0.5672
Epoch 00001: saving model to checkpoints\weights.01.hdf5
300/804 [==========>...................] - ETA: 1s - loss: 0.6714 - accuracy: 0.5817
Epoch 00001: saving model to checkpoints\weights.01.hdf5
378/804 [=============>................] - ETA: 0s - loss: 0.6678 - accuracy: 0.5872
Epoch 00001: saving model to checkpoints\weights.01.hdf5
476/804 [================>.............] - ETA: 0s - loss: 0.6644 - accuracy: 0.5925
Epoch 00001: saving model to checkpoints\weights.01.hdf5
575/804 [====================>.........] - ETA: 0s - loss: 0.6617 - accuracy: 0.5968
Epoch 00001: saving model to checkpoints\weights.01.hdf5
697/804 [=========================>....] - ETA: 0s - loss: 0.6592 - accuracy: 0.6008
Epoch 00001: saving model to checkpoints\weights.01

541/804 [===================>..........] - ETA: 0s - loss: 0.6379 - accuracy: 0.6229
Epoch 00007: saving model to checkpoints\weights.07.hdf5
636/804 [======================>.......] - ETA: 0s - loss: 0.6373 - accuracy: 0.6239
Epoch 00007: saving model to checkpoints\weights.07.hdf5
752/804 [===========================>..] - ETA: 0s - loss: 0.6368 - accuracy: 0.6249
Epoch 00007: saving model to checkpoints\weights.07.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6366 - accuracy: 0.6253
Epoch 8/100
 63/804 [=>............................] - ETA: 1s - loss: 0.6244 - accuracy: 0.6379
Epoch 00008: saving model to checkpoints\weights.08.hdf5
138/804 [====>.........................] - ETA: 1s - loss: 0.6267 - accuracy: 0.6379
Epoch 00008: saving model to checkpoints\weights.08.hdf5
250/804 [========>.....................] - ETA: 1s - loss: 0.6291 - accuracy: 0.6369
Epoch 00008: saving model to checkpoints\weights.08.hdf5
343/804 [===========>..................] - ETA:

236/804 [=======>......................] - ETA: 1s - loss: 0.6333 - accuracy: 0.6374
Epoch 00014: saving model to checkpoints\weights.14.hdf5
312/804 [==========>...................] - ETA: 1s - loss: 0.6324 - accuracy: 0.6376
Epoch 00014: saving model to checkpoints\weights.14.hdf5
414/804 [==============>...............] - ETA: 0s - loss: 0.6317 - accuracy: 0.6378
Epoch 00014: saving model to checkpoints\weights.14.hdf5
523/804 [==================>...........] - ETA: 0s - loss: 0.6313 - accuracy: 0.6381
Epoch 00014: saving model to checkpoints\weights.14.hdf5
629/804 [======================>.......] - ETA: 0s - loss: 0.6312 - accuracy: 0.6379
Epoch 00014: saving model to checkpoints\weights.14.hdf5
723/804 [=========================>....] - ETA: 0s - loss: 0.6313 - accuracy: 0.6376
Epoch 00014: saving model to checkpoints\weights.14.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6313 - accuracy: 0.6374
Epoch 15/100
 44/804 [>.............................] - ETA

684/804 [========================>.....] - ETA: 0s - loss: 0.6284 - accuracy: 0.6379
Epoch 00020: saving model to checkpoints\weights.20.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6289 - accuracy: 0.6373
Epoch 21/100
  1/804 [..............................] - ETA: 1s - loss: 0.6430 - accuracy: 0.6562
Epoch 00021: saving model to checkpoints\weights.21.hdf5
 81/804 [==>...........................] - ETA: 2s - loss: 0.6266 - accuracy: 0.6492
Epoch 00021: saving model to checkpoints\weights.21.hdf5
190/804 [======>.......................] - ETA: 1s - loss: 0.6282 - accuracy: 0.6402
Epoch 00021: saving model to checkpoints\weights.21.hdf5
288/804 [=========>....................] - ETA: 1s - loss: 0.6283 - accuracy: 0.6394
Epoch 00021: saving model to checkpoints\weights.21.hdf5
393/804 [=============>................] - ETA: 0s - loss: 0.6282 - accuracy: 0.6400
Epoch 00021: saving model to checkpoints\weights.21.hdf5
496/804 [=================>............] - ETA

389/804 [=============>................] - ETA: 0s - loss: 0.6322 - accuracy: 0.6337
Epoch 00027: saving model to checkpoints\weights.27.hdf5
488/804 [=================>............] - ETA: 0s - loss: 0.6318 - accuracy: 0.6341
Epoch 00027: saving model to checkpoints\weights.27.hdf5
552/804 [===================>..........] - ETA: 0s - loss: 0.6317 - accuracy: 0.6343
Epoch 00027: saving model to checkpoints\weights.27.hdf5
657/804 [=======================>......] - ETA: 0s - loss: 0.6315 - accuracy: 0.6343
Epoch 00027: saving model to checkpoints\weights.27.hdf5
771/804 [===========================>..] - ETA: 0s - loss: 0.6313 - accuracy: 0.6346
Epoch 00027: saving model to checkpoints\weights.27.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.6313 - accuracy: 0.6346
Epoch 28/100
 83/804 [==>...........................] - ETA: 0s - loss: 0.6326 - accuracy: 0.6426
Epoch 00028: saving model to checkpoints\weights.28.hdf5
159/804 [====>.........................] - ETA

...

287/804 [=========>....................] - ETA: 1s - loss: 0.6232 - accuracy: 0.6442
Epoch 00073: saving model to checkpoints\weights.73.hdf5
407/804 [==============>...............] - ETA: 1s - loss: 0.6240 - accuracy: 0.6450
Epoch 00073: saving model to checkpoints\weights.73.hdf5
479/804 [================>.............] - ETA: 1s - loss: 0.6244 - accuracy: 0.6454
Epoch 00073: saving model to checkpoints\weights.73.hdf5
609/804 [=====================>........] - ETA: 0s - loss: 0.6248 - accuracy: 0.6454
Epoch 00073: saving model to checkpoints\weights.73.hdf5
686/804 [========================>.....] - ETA: 0s - loss: 0.6251 - accuracy: 0.6449
Epoch 00073: saving model to checkpoints\weights.73.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.6255 - accuracy: 0.6442
Epoch 74/100
  1/804 [..............................] - ETA: 1s - loss: 0.5803 - accuracy: 0.7500
Epoch 00074: saving model to checkpoints\weights.74.hdf5
 66/804 [=>............................] - ETA

781/804 [============================>.] - ETA: 0s - loss: 0.6271 - accuracy: 0.6414
Epoch 00079: saving model to checkpoints\weights.79.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.6271 - accuracy: 0.6413
Epoch 80/100
 42/804 [>.............................] - ETA: 0s - loss: 0.6193 - accuracy: 0.6380
Epoch 00080: saving model to checkpoints\weights.80.hdf5
166/804 [=====>........................] - ETA: 0s - loss: 0.6236 - accuracy: 0.6337
Epoch 00080: saving model to checkpoints\weights.80.hdf5
257/804 [========>.....................] - ETA: 1s - loss: 0.6243 - accuracy: 0.6346
Epoch 00080: saving model to checkpoints\weights.80.hdf5
361/804 [============>.................] - ETA: 0s - loss: 0.6256 - accuracy: 0.6345
Epoch 00080: saving model to checkpoints\weights.80.hdf5
461/804 [================>.............] - ETA: 0s - loss: 0.6262 - accuracy: 0.6347
Epoch 00080: saving model to checkpoints\weights.80.hdf5
584/804 [====================>.........] - ETA

430/804 [===============>..............] - ETA: 0s - loss: 0.6254 - accuracy: 0.6405
Epoch 00086: saving model to checkpoints\weights.86.hdf5
524/804 [==================>...........] - ETA: 0s - loss: 0.6256 - accuracy: 0.6409
Epoch 00086: saving model to checkpoints\weights.86.hdf5
649/804 [=======================>......] - ETA: 0s - loss: 0.6258 - accuracy: 0.6411
Epoch 00086: saving model to checkpoints\weights.86.hdf5
755/804 [===========================>..] - ETA: 0s - loss: 0.6260 - accuracy: 0.6410
Epoch 00086: saving model to checkpoints\weights.86.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.6261 - accuracy: 0.6409
Epoch 87/100
 41/804 [>.............................] - ETA: 0s - loss: 0.6521 - accuracy: 0.6012
Epoch 00087: saving model to checkpoints\weights.87.hdf5
119/804 [===>..........................] - ETA: 1s - loss: 0.6396 - accuracy: 0.6200
Epoch 00087: saving model to checkpoints\weights.87.hdf5
227/804 [=======>......................] - ETA

111/804 [===>..........................] - ETA: 2s - loss: 0.6328 - accuracy: 0.6331
Epoch 00093: saving model to checkpoints\weights.93.hdf5
227/804 [=======>......................] - ETA: 1s - loss: 0.6310 - accuracy: 0.6371
Epoch 00093: saving model to checkpoints\weights.93.hdf5
298/804 [==========>...................] - ETA: 1s - loss: 0.6307 - accuracy: 0.6373
Epoch 00093: saving model to checkpoints\weights.93.hdf5
398/804 [=============>................] - ETA: 1s - loss: 0.6305 - accuracy: 0.6374
Epoch 00093: saving model to checkpoints\weights.93.hdf5
517/804 [==================>...........] - ETA: 0s - loss: 0.6304 - accuracy: 0.6374
Epoch 00093: saving model to checkpoints\weights.93.hdf5
615/804 [=====================>........] - ETA: 0s - loss: 0.6303 - accuracy: 0.6374
Epoch 00093: saving model to checkpoints\weights.93.hdf5
719/804 [=========================>....] - ETA: 0s - loss: 0.6301 - accuracy: 0.6376
Epoch 00093: saving model to checkpoints\weights.93.hdf5
804/80

587/804 [====================>.........] - ETA: 0s - loss: 0.6320 - accuracy: 0.6305
Epoch 00099: saving model to checkpoints\weights.99.hdf5
679/804 [========================>.....] - ETA: 0s - loss: 0.6315 - accuracy: 0.6313
Epoch 00099: saving model to checkpoints\weights.99.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.6309 - accuracy: 0.6323
Epoch 100/100
  1/804 [..............................] - ETA: 1s - loss: 0.5208 - accuracy: 0.7188
Epoch 00100: saving model to checkpoints\weights.100.hdf5
 61/804 [=>............................] - ETA: 2s - loss: 0.6164 - accuracy: 0.6456
Epoch 00100: saving model to checkpoints\weights.100.hdf5
179/804 [=====>........................] - ETA: 1s - loss: 0.6211 - accuracy: 0.6421
Epoch 00100: saving model to checkpoints\weights.100.hdf5
293/804 [=========>....................] - ETA: 1s - loss: 0.6231 - accuracy: 0.6403
Epoch 00100: saving model to checkpoints\weights.100.hdf5
388/804 [=============>................] 

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6348 - accuracy: 0.6364
Loss: 0.6348184943199158, Accuracy: 0.6363848447799683


In [17]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization2.h5")

### Optimization Attempt with New Model

In [18]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_new.fit(X_train_scaled,y_train,epochs=200,callbacks=[cp_callback])

Epoch 1/200
 76/804 [=>............................] - ETA: 0s - loss: 0.6905 - accuracy: 0.5282
Epoch 00001: saving model to checkpoints\weights.01.hdf5
178/804 [=====>........................] - ETA: 1s - loss: 0.6767 - accuracy: 0.5691
Epoch 00001: saving model to checkpoints\weights.01.hdf5
271/804 [=========>....................] - ETA: 1s - loss: 0.6701 - accuracy: 0.5834
Epoch 00001: saving model to checkpoints\weights.01.hdf5
363/804 [============>.................] - ETA: 1s - loss: 0.6655 - accuracy: 0.5913
Epoch 00001: saving model to checkpoints\weights.01.hdf5
465/804 [================>.............] - ETA: 0s - loss: 0.6615 - accuracy: 0.5974
Epoch 00001: saving model to checkpoints\weights.01.hdf5
566/804 [====================>.........] - ETA: 0s - loss: 0.6587 - accuracy: 0.6014
Epoch 00001: saving model to checkpoints\weights.01.hdf5
680/804 [========================>.....] - ETA: 0s - loss: 0.6566 - accuracy: 0.6044
Epoch 00001: saving model to checkpoints\weights.01

552/804 [===================>..........] - ETA: 0s - loss: 0.6330 - accuracy: 0.6360
Epoch 00007: saving model to checkpoints\weights.07.hdf5
643/804 [======================>.......] - ETA: 0s - loss: 0.6331 - accuracy: 0.6362
Epoch 00007: saving model to checkpoints\weights.07.hdf5
742/804 [==========================>...] - ETA: 0s - loss: 0.6331 - accuracy: 0.6362
Epoch 00007: saving model to checkpoints\weights.07.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6331 - accuracy: 0.6362
Epoch 8/200
 43/804 [>.............................] - ETA: 0s - loss: 0.6207 - accuracy: 0.6717
Epoch 00008: saving model to checkpoints\weights.08.hdf5
139/804 [====>.........................] - ETA: 1s - loss: 0.6247 - accuracy: 0.6492
Epoch 00008: saving model to checkpoints\weights.08.hdf5
252/804 [========>.....................] - ETA: 1s - loss: 0.6284 - accuracy: 0.6426
Epoch 00008: saving model to checkpoints\weights.08.hdf5
333/804 [===========>..................] - ETA:

135/804 [====>.........................] - ETA: 1s - loss: 0.6330 - accuracy: 0.6359
Epoch 00014: saving model to checkpoints\weights.14.hdf5
219/804 [=======>......................] - ETA: 1s - loss: 0.6308 - accuracy: 0.6371
Epoch 00014: saving model to checkpoints\weights.14.hdf5
315/804 [==========>...................] - ETA: 1s - loss: 0.6299 - accuracy: 0.6377
Epoch 00014: saving model to checkpoints\weights.14.hdf5
427/804 [==============>...............] - ETA: 0s - loss: 0.6293 - accuracy: 0.6380
Epoch 00014: saving model to checkpoints\weights.14.hdf5
520/804 [==================>...........] - ETA: 0s - loss: 0.6294 - accuracy: 0.6377
Epoch 00014: saving model to checkpoints\weights.14.hdf5
611/804 [=====================>........] - ETA: 0s - loss: 0.6297 - accuracy: 0.6374
Epoch 00014: saving model to checkpoints\weights.14.hdf5
743/804 [==========================>...] - ETA: 0s - loss: 0.6300 - accuracy: 0.6371
Epoch 00014: saving model to checkpoints\weights.14.hdf5
804/80

611/804 [=====================>........] - ETA: 0s - loss: 0.6333 - accuracy: 0.6335
Epoch 00020: saving model to checkpoints\weights.20.hdf5
705/804 [=========================>....] - ETA: 0s - loss: 0.6329 - accuracy: 0.6340
Epoch 00020: saving model to checkpoints\weights.20.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.6326 - accuracy: 0.6343
Epoch 21/200
  1/804 [..............................] - ETA: 1s - loss: 0.6796 - accuracy: 0.5312
Epoch 00021: saving model to checkpoints\weights.21.hdf5
 95/804 [==>...........................] - ETA: 2s - loss: 0.6292 - accuracy: 0.6365 ETA: 4s - loss: 0.6317 - accuracy: 0.
Epoch 00021: saving model to checkpoints\weights.21.hdf5
216/804 [=======>......................] - ETA: 1s - loss: 0.6279 - accuracy: 0.6392
Epoch 00021: saving model to checkpoints\weights.21.hdf5
320/804 [==========>...................] - ETA: 1s - loss: 0.6292 - accuracy: 0.6379
Epoch 00021: saving model to checkpoints\weights.21.hdf5
387/804 

160/804 [====>.........................] - ETA: 1s - loss: 0.6289 - accuracy: 0.6377
Epoch 00027: saving model to checkpoints\weights.27.hdf5
265/804 [========>.....................] - ETA: 1s - loss: 0.6293 - accuracy: 0.6393
Epoch 00027: saving model to checkpoints\weights.27.hdf5
362/804 [============>.................] - ETA: 1s - loss: 0.6289 - accuracy: 0.6406
Epoch 00027: saving model to checkpoints\weights.27.hdf5
468/804 [================>.............] - ETA: 0s - loss: 0.6287 - accuracy: 0.6412
Epoch 00027: saving model to checkpoints\weights.27.hdf5
568/804 [====================>.........] - ETA: 0s - loss: 0.6287 - accuracy: 0.6409
Epoch 00027: saving model to checkpoints\weights.27.hdf5
685/804 [========================>.....] - ETA: 0s - loss: 0.6289 - accuracy: 0.6405
Epoch 00027: saving model to checkpoints\weights.27.hdf5
763/804 [===========================>..] - ETA: 0s - loss: 0.6289 - accuracy: 0.6403
Epoch 00027: saving model to checkpoints\weights.27.hdf5
804/80

...

309/804 [==========>...................] - ETA: 1s - loss: 0.6294 - accuracy: 0.6362
Epoch 00171: saving model to checkpoints\weights.171.hdf5
389/804 [=============>................] - ETA: 1s - loss: 0.6292 - accuracy: 0.6365
Epoch 00171: saving model to checkpoints\weights.171.hdf5
484/804 [=================>............] - ETA: 0s - loss: 0.6290 - accuracy: 0.6367
Epoch 00171: saving model to checkpoints\weights.171.hdf5
604/804 [=====================>........] - ETA: 0s - loss: 0.6285 - accuracy: 0.6375
Epoch 00171: saving model to checkpoints\weights.171.hdf5
693/804 [========================>.....] - ETA: 0s - loss: 0.6282 - accuracy: 0.6382
Epoch 00171: saving model to checkpoints\weights.171.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6280 - accuracy: 0.6387
Epoch 172/200
  1/804 [..............................] - ETA: 2s - loss: 0.6509 - accuracy: 0.5938
Epoch 00172: saving model to checkpoints\weights.172.hdf5
113/804 [===>..........................

Epoch 00177: saving model to checkpoints\weights.177.hdf5
758/804 [===========================>..] - ETA: 0s - loss: 0.6269 - accuracy: 0.6365
Epoch 00177: saving model to checkpoints\weights.177.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6269 - accuracy: 0.6367
Epoch 178/200
 83/804 [==>...........................] - ETA: 0s - loss: 0.6259 - accuracy: 0.6384
Epoch 00178: saving model to checkpoints\weights.178.hdf5
191/804 [======>.......................] - ETA: 0s - loss: 0.6246 - accuracy: 0.6381
Epoch 00178: saving model to checkpoints\weights.178.hdf5
257/804 [========>.....................] - ETA: 0s - loss: 0.6250 - accuracy: 0.6377
Epoch 00178: saving model to checkpoints\weights.178.hdf5
351/804 [============>.................] - ETA: 1s - loss: 0.6254 - accuracy: 0.6376
Epoch 00178: saving model to checkpoints\weights.178.hdf5
465/804 [================>.............] - ETA: 0s - loss: 0.6258 - accuracy: 0.6378
Epoch 00178: saving model to checkpoint

Epoch 00184: saving model to checkpoints\weights.184.hdf5
348/804 [===========>..................] - ETA: 0s - loss: 0.6227 - accuracy: 0.6531
Epoch 00184: saving model to checkpoints\weights.184.hdf5
446/804 [===============>..............] - ETA: 0s - loss: 0.6233 - accuracy: 0.6503
Epoch 00184: saving model to checkpoints\weights.184.hdf5
537/804 [===================>..........] - ETA: 0s - loss: 0.6235 - accuracy: 0.6488
Epoch 00184: saving model to checkpoints\weights.184.hdf5
649/804 [=======================>......] - ETA: 0s - loss: 0.6238 - accuracy: 0.6475
Epoch 00184: saving model to checkpoints\weights.184.hdf5
744/804 [==========================>...] - ETA: 0s - loss: 0.6239 - accuracy: 0.6467
Epoch 00184: saving model to checkpoints\weights.184.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6241 - accuracy: 0.6463
Epoch 185/200
 43/804 [>.............................] - ETA: 0s - loss: 0.6203 - accuracy: 0.6459
Epoch 00185: saving model to checkpoint

737/804 [==========================>...] - ETA: 0s - loss: 0.6245 - accuracy: 0.6423
Epoch 00190: saving model to checkpoints\weights.190.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6247 - accuracy: 0.6421
Epoch 191/200
 40/804 [>.............................] - ETA: 0s - loss: 0.6287 - accuracy: 0.6223
Epoch 00191: saving model to checkpoints\weights.191.hdf5
122/804 [===>..........................] - ETA: 1s - loss: 0.6285 - accuracy: 0.6298
Epoch 00191: saving model to checkpoints\weights.191.hdf5
226/804 [=======>......................] - ETA: 1s - loss: 0.6266 - accuracy: 0.6349
Epoch 00191: saving model to checkpoints\weights.191.hdf5
326/804 [===========>..................] - ETA: 1s - loss: 0.6254 - accuracy: 0.6364
Epoch 00191: saving model to checkpoints\weights.191.hdf5
417/804 [==============>...............] - ETA: 0s - loss: 0.6250 - accuracy: 0.6374
Epoch 00191: saving model to checkpoints\weights.191.hdf5
513/804 [==================>...........

295/804 [==========>...................] - ETA: 1s - loss: 0.6278 - accuracy: 0.6433
Epoch 00197: saving model to checkpoints\weights.197.hdf5
385/804 [=============>................] - ETA: 1s - loss: 0.6277 - accuracy: 0.6426
Epoch 00197: saving model to checkpoints\weights.197.hdf5
493/804 [=================>............] - ETA: 0s - loss: 0.6274 - accuracy: 0.6423
Epoch 00197: saving model to checkpoints\weights.197.hdf5
579/804 [====================>.........] - ETA: 0s - loss: 0.6271 - accuracy: 0.6423
Epoch 00197: saving model to checkpoints\weights.197.hdf5
681/804 [========================>.....] - ETA: 0s - loss: 0.6268 - accuracy: 0.6421
Epoch 00197: saving model to checkpoints\weights.197.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6267 - accuracy: 0.6418
Epoch 198/200
  1/804 [..............................] - ETA: 1s - loss: 0.5653 - accuracy: 0.7500
Epoch 00198: saving model to checkpoints\weights.198.hdf5
 73/804 [=>............................

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6424 - accuracy: 0.6374
Loss: 0.6423842906951904, Accuracy: 0.6374344229698181
